In [320]:
import glob
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

KPI_data = {os.path.splitext(os.path.basename(file_name))[0] : pd.read_csv(file_name) for file_name in glob.glob('original_data/*')}

In [321]:
# for dataset_name in KPI_data:
#     print(dataset_name, KPI_data[dataset_name].columns[-1])
    
# dataset_country_name = [set(KPI_data[_].country) for _ in KPI_data]
# print(dataset_country_name[0].intersection(*dataset_country_name[1:]))
    

In [322]:
# global_internet_users = pd.read_csv('original_data/global_internet_users.csv')

In [323]:
# global_internet_users.describe()

In [324]:
# unique_entietes = global_internet_users['Entity'].unique()
# unique_entietes.__len__()

In [325]:
# countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
# countries.head()

In [326]:
# gpd_global_internet_users = countries.set_index('iso_a3').join(global_internet_users.set_index('Code'))

In [327]:
# def plot_global_map(df: pd.DataFrame, col: 'str', year: int):
#     fig, ax = plt.subplots(figsize=(20, 10))
#     df = df[df['Year'] == year]
#     df.plot(column=col, cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8')
#     vmin = df[col].min()
#     vmax = df[col].max()
#     sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=vmin, vmax=vmax))
#     sm.set_array([])
#     plt.title(year)
#     plt.tight_layout()
#     plt.colorbar(sm)

In [328]:
# plot_global_map(gpd_global_internet_users, 'Internet Users(%)', 2020)

In [329]:
# plot_global_map(gpd_global_internet_users, 'Cellular Subscription', 2020)

In [330]:
# plot_global_map(gpd_global_internet_users, 'Broadband Subscription', 2020)

# Data Imputation

In [331]:
child_mortality_df = KPI_data["child_mortality_0_5_year_olds_dying_per_1000_born"]
coal_consumption_df = KPI_data["coal_consumption_per_cap"]
food_supply_df = KPI_data["food_supply_kilocalories_per_person_and_day"]
med_beds_df = KPI_data["sh_med_beds_zs"]

years_range = [str(i) for i in range(2000, 2020)]

### child_mortality_0_5_year_olds_dying_per_1000_born

In [332]:
child_mortality_df.columns

Index(['country', '1800', '1801', '1802', '1803', '1804', '1805', '1806',
       '1807', '1808',
       ...
       '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099',
       '2100'],
      dtype='object', length=302)

In [333]:
child_mortality_df = child_mortality_df[["country"] + years_range]
child_mortality_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,129.00,125.00,121.00,117.00,113.00,109.00,104.00,100.00,96.00,...,88.00,84.10,80.50,77.00,73.50,70.40,67.50,64.70,62.30,60.10
1,Angola,206.00,200.00,193.00,185.00,176.00,167.00,157.00,148.00,138.00,...,120.00,113.00,105.00,98.40,92.90,88.10,84.00,80.40,77.20,74.50
2,Albania,25.90,24.50,23.10,21.80,20.40,19.20,17.90,16.70,15.50,...,13.30,12.30,11.40,10.70,10.10,9.60,9.27,9.02,8.82,8.68
3,Andorra,6.41,6.16,5.93,5.71,5.49,5.27,5.05,4.84,4.62,...,4.18,3.98,3.79,3.61,3.43,3.27,3.14,3.01,2.89,2.87
4,United Arab Emirates,11.20,10.90,10.60,10.30,10.00,9.73,9.44,9.18,8.93,...,8.48,8.28,8.13,8.00,7.91,7.83,7.76,7.68,7.59,7.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Samoa,21.10,20.40,19.90,19.50,19.20,19.00,18.90,18.90,18.90,...,18.70,18.50,18.30,17.90,17.60,17.20,16.70,16.30,15.80,15.40
193,Yemen,94.90,90.30,85.60,81.10,76.70,72.50,68.40,64.60,61.00,...,56.00,55.10,55.00,55.00,55.00,55.00,55.00,55.00,55.00,54.90
194,South Africa,73.90,75.80,77.40,79.10,79.40,78.50,76.00,71.00,64.80,...,52.50,46.70,43.30,41.20,39.50,37.70,36.60,35.30,33.80,32.90
195,Zambia,162.00,153.00,142.00,130.00,119.00,110.00,101.00,95.40,90.40,...,80.30,77.20,74.30,70.70,66.90,64.60,62.00,59.40,57.80,56.10


In [334]:
child_mortality_df[child_mortality_df.isna().any(axis=1)]

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019


In [335]:
child_mortality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  197 non-null    object 
 1   2000     197 non-null    float64
 2   2001     197 non-null    float64
 3   2002     197 non-null    float64
 4   2003     197 non-null    float64
 5   2004     197 non-null    float64
 6   2005     197 non-null    float64
 7   2006     197 non-null    float64
 8   2007     197 non-null    float64
 9   2008     197 non-null    float64
 10  2009     197 non-null    float64
 11  2010     197 non-null    float64
 12  2011     197 non-null    float64
 13  2012     197 non-null    float64
 14  2013     197 non-null    float64
 15  2014     197 non-null    float64
 16  2015     197 non-null    float64
 17  2016     197 non-null    float64
 18  2017     197 non-null    float64
 19  2018     197 non-null    float64
 20  2019     197 non-null    float64
dtypes: float64(20), 

### coal_consumption_per_cap

In [336]:
coal_consumption_df.columns

Index(['country', '1965', '1966', '1967', '1968', '1969', '1970', '1971',
       '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'],
      dtype='object')

In [337]:
coal_consumption_df = coal_consumption_df[["country"] + years_range]
coal_consumption_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United Arab Emirates,0,0.0056,0.02710,0.0299,0.0309,0.03170,0.05310,0.02200,0.04840,...,0.07700,0.04970,0.15100,0.19200,0.21400,0.18500,0.19700,0.22400,0.25000,0.25200
1,Argentina,0.0175,0.0142,0.01050,0.0126,0.0213,0.02420,0.02170,0.03110,0.03760,...,0.02830,0.03040,0.02820,0.03040,0.03210,0.03160,0.02410,0.02450,0.02660,0.01220
2,Australia,2.68,2.62,2.65000,2.7,2.78,2.72000,2.75000,2.67000,2.73000,...,2.36000,2.26000,2.08000,1.95000,1.90000,1.94000,1.91000,1.82000,1.77000,1.69000
3,Austria,0.453,0.466,0.47100,0.5,0.487,0.48400,0.48200,0.46400,0.45700,...,0.40100,0.41000,0.37900,0.38300,0.34800,0.37400,0.34400,0.35300,0.32100,0.33600
4,Azerbaijan,0,0.00037,0.00024,0.00036,0.00011,0.00038,0.00037,0.00057,0.00042,...,0.00061,0.00054,0.00040,0.00033,0.00019,0.00032,0.00013,0.00013,0.00012,0.00012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,United States,1.92,1.84,1.83000,1.84,1.84,1.85000,1.80000,1.81000,1.77000,...,1.61000,1.51000,1.32000,1.36000,1.35000,1.16000,1.05000,1.02000,0.97000,0.82300
75,Uzbekistan,0.0286,0.0307,0.03080,0.0213,0.0293,0.03220,0.03280,0.03390,0.03670,...,0.03590,0.03770,0.03950,0.05420,0.05080,0.04510,0.04150,0.04810,0.06250,0.05260
76,Venezuela,0.00546,0.00198,0.00073,0.00169,0,0.00141,0.00922,0.00509,0.00510,...,0.00701,0.00710,0.00719,0.00706,0.00656,0.00454,0.00413,0.00432,0.00398,0.00265
77,Vietnam,0.0587,0.0635,0.07180,0.0828,0.107,0.11200,0.06770,0.07340,0.13800,...,0.16500,0.19500,0.17900,0.18900,0.22700,0.28300,0.30300,0.30000,0.39700,0.51300


In [338]:
coal_consumption_df[coal_consumption_df.isna().any(axis=1)]

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019


In [339]:
coal_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  79 non-null     object 
 1   2000     79 non-null     object 
 2   2001     79 non-null     object 
 3   2002     79 non-null     float64
 4   2003     79 non-null     object 
 5   2004     79 non-null     object 
 6   2005     79 non-null     float64
 7   2006     79 non-null     float64
 8   2007     79 non-null     float64
 9   2008     79 non-null     float64
 10  2009     79 non-null     float64
 11  2010     79 non-null     float64
 12  2011     79 non-null     float64
 13  2012     79 non-null     float64
 14  2013     79 non-null     float64
 15  2014     79 non-null     float64
 16  2015     79 non-null     float64
 17  2016     79 non-null     float64
 18  2017     79 non-null     float64
 19  2018     79 non-null     float64
 20  2019     79 non-null     float64
dtypes: float64(16), ob

In [340]:
for i in range(1, 10):
    coal_consumption_df = coal_consumption_df.replace(str(i) + '0µ', str(i * 10**-5))

In [341]:
for year in [2000, 2001, 2003, 2004]:
    coal_consumption_df[str(year)] = coal_consumption_df[str(year)].astype("float64")

In [342]:
coal_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  79 non-null     object 
 1   2000     79 non-null     float64
 2   2001     79 non-null     float64
 3   2002     79 non-null     float64
 4   2003     79 non-null     float64
 5   2004     79 non-null     float64
 6   2005     79 non-null     float64
 7   2006     79 non-null     float64
 8   2007     79 non-null     float64
 9   2008     79 non-null     float64
 10  2009     79 non-null     float64
 11  2010     79 non-null     float64
 12  2011     79 non-null     float64
 13  2012     79 non-null     float64
 14  2013     79 non-null     float64
 15  2014     79 non-null     float64
 16  2015     79 non-null     float64
 17  2016     79 non-null     float64
 18  2017     79 non-null     float64
 19  2018     79 non-null     float64
 20  2019     79 non-null     float64
dtypes: float64(20), ob

### food_supply_kilocalories_per_person_and_day

In [343]:
food_supply_df.columns

Index(['country', '1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [344]:
food_supply_df = food_supply_df[["country"] + years_range[:-1]]
food_supply_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,1790.0,1740.0,1830.0,1890.0,1970.0,1950.0,1970.0,2050.0,2040.0,2080.0,2100.0,2110.0,2100.0,2090.0,2100.0,2040.0,2030.0,2050.0,2040.0
1,Angola,1790.0,1830.0,1920.0,1980.0,2030.0,2080.0,2120.0,2170.0,2250.0,2300.0,2350.0,2410.0,2380.0,2470.0,2420.0,2380.0,2340.0,2370.0,2390.0
2,Albania,2730.0,2800.0,2860.0,2770.0,2790.0,2870.0,2860.0,2860.0,2950.0,2990.0,3080.0,3130.0,3180.0,3190.0,3270.0,3280.0,3380.0,3390.0,3360.0
3,Netherlands Antilles,3080.0,3050.0,3070.0,3060.0,3080.0,3090.0,3090.0,3070.0,3080.0,3100.0,3110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Arab Emirates,3300.0,3320.0,3360.0,3340.0,3290.0,3210.0,3200.0,3190.0,3150.0,3170.0,3160.0,3210.0,3230.0,3280.0,3400.0,3410.0,3370.0,3340.0,3310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Yemen,2090.0,2070.0,2080.0,2070.0,2070.0,2090.0,2120.0,2160.0,2130.0,2160.0,2200.0,2210.0,2230.0,2220.0,2160.0,2170.0,2010.0,2070.0,2150.0
175,Yugoslavia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,South Africa,2890.0,2910.0,2910.0,2930.0,2940.0,2950.0,2930.0,2920.0,2920.0,2940.0,2990.0,3000.0,3010.0,3020.0,2980.0,2940.0,2920.0,2940.0,2900.0
177,Zambia,1870.0,1850.0,1850.0,1900.0,1870.0,1870.0,1840.0,1780.0,1800.0,1880.0,1900.0,1910.0,1920.0,1930.0,2010.0,2030.0,1990.0,1990.0,2000.0


In [345]:
food_supply_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  179 non-null    object 
 1   2000     174 non-null    float64
 2   2001     174 non-null    float64
 3   2002     174 non-null    float64
 4   2003     174 non-null    float64
 5   2004     174 non-null    float64
 6   2005     174 non-null    float64
 7   2006     175 non-null    float64
 8   2007     175 non-null    float64
 9   2008     175 non-null    float64
 10  2009     175 non-null    float64
 11  2010     175 non-null    float64
 12  2011     174 non-null    float64
 13  2012     174 non-null    float64
 14  2013     174 non-null    float64
 15  2014     172 non-null    float64
 16  2015     172 non-null    float64
 17  2016     172 non-null    float64
 18  2017     172 non-null    float64
 19  2018     172 non-null    float64
dtypes: float64(19), object(1)
memory usage: 28.1+ KB


In [346]:
food_supply_df[food_supply_df.isna().any(axis=1)]

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,Netherlands Antilles,3080.0,3050.0,3070.0,3060.0,3080.0,3090.0,3090.0,3070.0,3080.0,3100.0,3110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Bermuda,2650.0,2610.0,2580.0,2490.0,2460.0,2520.0,2590.0,2630.0,2700.0,2720.0,2680.0,2650.0,2650.0,2670.0,NaN,NaN,NaN,NaN,NaN
24,Brunei,2800.0,2880.0,2930.0,2980.0,3000.0,2980.0,2970.0,2920.0,2910.0,2890.0,2890.0,2940.0,2970.0,2990.0,NaN,NaN,NaN,NaN,NaN
29,Czechoslovakia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,Montenegro,NaN,NaN,NaN,NaN,NaN,NaN,3280.0,3410.0,3480.0,3490.0,3460.0,3480.0,3370.0,3490.0,3380.0,3420.0,3480.0,3500.0,3500.0
139,Serbia and Montenegro,2650.0,2610.0,2630.0,2700.0,2700.0,2700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,Serbia,NaN,NaN,NaN,NaN,NaN,NaN,2750.0,2710.0,2720.0,2760.0,2730.0,2730.0,2720.0,2730.0,2810.0,2790.0,2810.0,2820.0,2830.0
167,USSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,Yugoslavia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### If there are gaps in values, they are not isolated

In [347]:
food_supply_df = food_supply_df.dropna()

In [348]:
food_supply_df[food_supply_df.isna().any(axis=1)]

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


In [349]:
food_supply_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 178
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  170 non-null    object 
 1   2000     170 non-null    float64
 2   2001     170 non-null    float64
 3   2002     170 non-null    float64
 4   2003     170 non-null    float64
 5   2004     170 non-null    float64
 6   2005     170 non-null    float64
 7   2006     170 non-null    float64
 8   2007     170 non-null    float64
 9   2008     170 non-null    float64
 10  2009     170 non-null    float64
 11  2010     170 non-null    float64
 12  2011     170 non-null    float64
 13  2012     170 non-null    float64
 14  2013     170 non-null    float64
 15  2014     170 non-null    float64
 16  2015     170 non-null    float64
 17  2016     170 non-null    float64
 18  2017     170 non-null    float64
 19  2018     170 non-null    float64
dtypes: float64(19), object(1)
memory usage: 27.9+ KB


In [350]:
food_supply_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,1790.0,1740.0,1830.0,1890.0,1970.0,1950.0,1970.0,2050.0,2040.0,2080.0,2100.0,2110.0,2100.0,2090.0,2100.0,2040.0,2030.0,2050.0,2040.0
1,Angola,1790.0,1830.0,1920.0,1980.0,2030.0,2080.0,2120.0,2170.0,2250.0,2300.0,2350.0,2410.0,2380.0,2470.0,2420.0,2380.0,2340.0,2370.0,2390.0
2,Albania,2730.0,2800.0,2860.0,2770.0,2790.0,2870.0,2860.0,2860.0,2950.0,2990.0,3080.0,3130.0,3180.0,3190.0,3270.0,3280.0,3380.0,3390.0,3360.0
4,United Arab Emirates,3300.0,3320.0,3360.0,3340.0,3290.0,3210.0,3200.0,3190.0,3150.0,3170.0,3160.0,3210.0,3230.0,3280.0,3400.0,3410.0,3370.0,3340.0,3310.0
5,Argentina,3260.0,3210.0,2980.0,3010.0,3030.0,3110.0,3110.0,3150.0,3160.0,3130.0,3160.0,3160.0,3200.0,3230.0,3250.0,3310.0,3250.0,3270.0,3310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Samoa,2770.0,2820.0,2820.0,2830.0,2880.0,2890.0,2860.0,2850.0,2910.0,2880.0,2910.0,2880.0,2960.0,2960.0,2970.0,2970.0,3000.0,3040.0,3030.0
174,Yemen,2090.0,2070.0,2080.0,2070.0,2070.0,2090.0,2120.0,2160.0,2130.0,2160.0,2200.0,2210.0,2230.0,2220.0,2160.0,2170.0,2010.0,2070.0,2150.0
176,South Africa,2890.0,2910.0,2910.0,2930.0,2940.0,2950.0,2930.0,2920.0,2920.0,2940.0,2990.0,3000.0,3010.0,3020.0,2980.0,2940.0,2920.0,2940.0,2900.0
177,Zambia,1870.0,1850.0,1850.0,1900.0,1870.0,1870.0,1840.0,1780.0,1800.0,1880.0,1900.0,1910.0,1920.0,1930.0,2010.0,2030.0,1990.0,1990.0,2000.0


In [351]:
import pmdarima as pm
import warnings
warnings.filterwarnings("ignore")

In [352]:
df = food_supply_df.drop(["country"], axis = 1)


In [353]:
pred = []
for _, row in df.iterrows():
    auto_arima=pm.auto_arima(row, start_p = 0, start_q = 0, max_p = 12, max_q = 12, m = 12, start_P = 0, start_Q = 0, seasonal = False, error_action = 'ignore', suppress_warnings = True, stepwise = True)
    prediction = pd.DataFrame(auto_arima.predict(n_periods=1))
    pred.append(prediction)

In [354]:
for i in range(len(pred)):
    pred[i] = float(int(pred[i].iat[0, 0]))

In [355]:
food_supply_df["2019"] = pred
food_supply_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,1790.0,1740.0,1830.0,1890.0,1970.0,1950.0,1970.0,2050.0,2040.0,...,2100.0,2110.0,2100.0,2090.0,2100.0,2040.0,2030.0,2050.0,2040.0,2040.0
1,Angola,1790.0,1830.0,1920.0,1980.0,2030.0,2080.0,2120.0,2170.0,2250.0,...,2350.0,2410.0,2380.0,2470.0,2420.0,2380.0,2340.0,2370.0,2390.0,2423.0
2,Albania,2730.0,2800.0,2860.0,2770.0,2790.0,2870.0,2860.0,2860.0,2950.0,...,3080.0,3130.0,3180.0,3190.0,3270.0,3280.0,3380.0,3390.0,3360.0,3395.0
4,United Arab Emirates,3300.0,3320.0,3360.0,3340.0,3290.0,3210.0,3200.0,3190.0,3150.0,...,3160.0,3210.0,3230.0,3280.0,3400.0,3410.0,3370.0,3340.0,3310.0,3285.0
5,Argentina,3260.0,3210.0,2980.0,3010.0,3030.0,3110.0,3110.0,3150.0,3160.0,...,3160.0,3160.0,3200.0,3230.0,3250.0,3310.0,3250.0,3270.0,3310.0,3310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Samoa,2770.0,2820.0,2820.0,2830.0,2880.0,2890.0,2860.0,2850.0,2910.0,...,2910.0,2880.0,2960.0,2960.0,2970.0,2970.0,3000.0,3040.0,3030.0,3055.0
174,Yemen,2090.0,2070.0,2080.0,2070.0,2070.0,2090.0,2120.0,2160.0,2130.0,...,2200.0,2210.0,2230.0,2220.0,2160.0,2170.0,2010.0,2070.0,2150.0,2142.0
176,South Africa,2890.0,2910.0,2910.0,2930.0,2940.0,2950.0,2930.0,2920.0,2920.0,...,2990.0,3000.0,3010.0,3020.0,2980.0,2940.0,2920.0,2940.0,2900.0,2892.0
177,Zambia,1870.0,1850.0,1850.0,1900.0,1870.0,1870.0,1840.0,1780.0,1800.0,...,1900.0,1910.0,1920.0,1930.0,2010.0,2030.0,1990.0,1990.0,2000.0,2000.0


### sh_med_beds_zs

In [356]:
med_beds_df.columns

Index(['country', '1960', '1961', '1962', '1963', '1964', '1965', '1966',
       '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [357]:
med_beds_df = med_beds_df [["country"] + years_range]
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.30,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,...,0.43,0.44,0.53,0.53,0.50,0.50,0.50,0.39,NaN,NaN
1,Angola,NaN,NaN,NaN,NaN,NaN,0.80,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,NaN,...,2.99,2.88,2.88,2.89,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,3.20,2.59,NaN,3.30,NaN,2.70,2.60,2.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,...,1.93,1.07,1.07,1.15,0.01,1.10,1.36,1.38,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Samoa,3.30,NaN,1.50,2.04,NaN,1.00,NaN,1.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Yemen,0.59,0.59,0.59,0.59,0.59,0.61,0.70,0.70,0.70,...,0.72,0.70,0.72,0.67,0.71,0.71,0.71,0.71,NaN,NaN
198,South Africa,NaN,NaN,3.10,NaN,2.87,2.80,NaN,2.41,2.39,...,2.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,Zambia,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,1.90,...,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [358]:
med_beds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  201 non-null    object 
 1   2000     99 non-null     float64
 2   2001     103 non-null    float64
 3   2002     113 non-null    float64
 4   2003     108 non-null    float64
 5   2004     109 non-null    float64
 6   2005     133 non-null    float64
 7   2006     134 non-null    float64
 8   2007     119 non-null    float64
 9   2008     114 non-null    float64
 10  2009     127 non-null    float64
 11  2010     139 non-null    float64
 12  2011     134 non-null    float64
 13  2012     118 non-null    float64
 14  2013     112 non-null    float64
 15  2014     112 non-null    float64
 16  2015     107 non-null    float64
 17  2016     104 non-null    float64
 18  2017     98 non-null     float64
 19  2018     36 non-null     float64
 20  2019     8 non-null      float64
dtypes: float64(20), 

In [359]:
med_beds_df = med_beds_df [["country"] + years_range[:-2]]
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,0.30,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,0.42,0.43,0.44,0.53,0.53,0.50,0.50,0.50,0.39
1,Angola,NaN,NaN,NaN,NaN,NaN,0.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,NaN,3.01,2.99,2.88,2.88,2.89,NaN,NaN,NaN,NaN
3,Andorra,3.20,2.59,NaN,3.30,NaN,2.70,2.60,2.60,NaN,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,1.93,1.93,1.07,1.07,1.15,0.01,1.10,1.36,1.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Samoa,3.30,NaN,1.50,2.04,NaN,1.00,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Yemen,0.59,0.59,0.59,0.59,0.59,0.61,0.70,0.70,0.70,0.70,0.72,0.70,0.72,0.67,0.71,0.71,0.71,0.71
198,South Africa,NaN,NaN,3.10,NaN,2.87,2.80,NaN,2.41,2.39,NaN,2.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,Zambia,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,1.90,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [360]:
med_beds_df = med_beds_df[med_beds_df.isna().sum(axis=1) < 9]
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,0.30,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,0.42,0.43,0.44,0.53,0.53,0.50,0.50,0.50,0.39
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,NaN,3.01,2.99,2.88,2.88,2.89,NaN,NaN,NaN,NaN
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,1.93,1.93,1.07,1.07,1.15,0.01,1.10,1.36,1.38
5,Argentina,4.10,NaN,NaN,NaN,NaN,4.00,NaN,NaN,NaN,NaN,4.50,4.39,4.59,4.78,4.84,4.96,4.95,4.99
6,Armenia,6.44,5.03,4.35,4.42,4.44,4.46,4.42,4.07,3.82,3.72,3.73,3.74,4.05,4.06,4.16,4.20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,United States,3.49,3.47,3.39,3.33,3.26,3.20,3.18,3.14,3.13,3.08,3.05,2.97,2.93,2.89,2.83,2.80,2.77,2.87
190,Uzbekistan,5.33,5.34,5.54,5.48,5.26,5.19,5.12,4.83,4.67,4.58,4.44,4.32,4.19,3.99,3.98,NaN,NaN,NaN
191,St. Vincent and the Grenadines,NaN,4.70,4.50,4.50,4.50,4.50,NaN,3.00,NaN,NaN,2.60,2.52,2.41,2.41,2.40,4.33,4.32,NaN
194,Vietnam,2.34,2.40,1.40,NaN,2.80,2.34,2.66,NaN,2.90,3.10,2.91,NaN,2.50,3.18,2.60,NaN,NaN,NaN


In [361]:
med_beds_df = med_beds_df.T
med_beds_df

,0,2,4,5,6,7,8,9,10,12,...,181,182,183,187,188,189,190,191,194,197
country,Afghanistan,Albania,United Arab Emirates,Argentina,Armenia,Antigua and Barbuda,Australia,Austria,Azerbaijan,Belgium,...,Trinidad and Tobago,Tunisia,Turkey,Ukraine,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,Vietnam,Yemen
2000,0.3,3.26,2.38,4.1,6.44,NaN,4.04,7.95,8.69,6.72,...,3.3,1.73,2.1,8.82,NaN,3.49,5.33,NaN,2.34,0.59
2001,0.39,3.26,2.28,NaN,5.03,2.6,3.95,7.85,8.51,6.62,...,3.4,1.73,2.16,8.79,NaN,3.47,5.34,4.7,2.4,0.59
2002,0.39,3.14,2.19,NaN,4.35,2.3,3.93,7.81,8.4,6.54,...,NaN,1.71,2.49,8.89,NaN,3.39,5.54,4.5,1.4,0.59
2003,0.39,3.07,2.19,NaN,4.42,2.5,3.97,7.73,8.27,6.53,...,3.3,1.71,2.48,8.81,1.9,3.33,5.48,4.5,NaN,0.59
2004,0.39,3.01,2.19,NaN,4.44,2.4,4.0,7.73,8.24,6.48,...,NaN,2.05,2.47,8.73,NaN,3.26,5.26,4.5,2.8,0.59
2005,0.42,3.08,2.19,4.0,4.46,2.4,3.9,7.69,8.21,6.44,...,2.6,1.76,2.5,8.68,2.4,3.2,5.19,4.5,2.34,0.61
2006,0.42,3.12,1.88,NaN,4.42,2.4,3.93,7.66,7.94,6.35,...,2.7,1.98,2.52,8.73,2.9,3.18,5.12,NaN,2.66,0.7
2007,0.42,3.09,1.88,NaN,4.07,2.0,NaN,7.75,7.93,6.27,...,2.7,1.76,2.54,8.72,2.9,3.14,4.83,3.0,NaN,0.7
2008,0.42,NaN,1.86,NaN,3.82,1.7,3.82,7.69,7.67,6.25,...,2.5,2.0,2.58,8.75,NaN,3.13,4.67,NaN,2.9,0.7


In [362]:
med_beds_df = med_beds_df.fillna(method='bfill')
med_beds_df = med_beds_df.fillna(method='ffill')

In [363]:
med_beds_df = med_beds_df.T
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,0.3,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,0.42,0.43,0.44,0.53,0.53,0.5,0.5,0.5,0.39
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,3.01,3.01,2.99,2.88,2.88,2.89,2.89,2.89,2.89,2.89
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,1.93,1.93,1.07,1.07,1.15,0.01,1.1,1.36,1.38
5,Argentina,4.1,4.0,4.0,4.0,4.0,4.0,4.5,4.5,4.5,4.5,4.5,4.39,4.59,4.78,4.84,4.96,4.95,4.99
6,Armenia,6.44,5.03,4.35,4.42,4.44,4.46,4.42,4.07,3.82,3.72,3.73,3.74,4.05,4.06,4.16,4.2,4.2,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,United States,3.49,3.47,3.39,3.33,3.26,3.2,3.18,3.14,3.13,3.08,3.05,2.97,2.93,2.89,2.83,2.8,2.77,2.87
190,Uzbekistan,5.33,5.34,5.54,5.48,5.26,5.19,5.12,4.83,4.67,4.58,4.44,4.32,4.19,3.99,3.98,3.98,3.98,3.98
191,St. Vincent and the Grenadines,4.7,4.7,4.5,4.5,4.5,4.5,3.0,3.0,2.6,2.6,2.6,2.52,2.41,2.41,2.4,4.33,4.32,4.32
194,Vietnam,2.34,2.4,1.4,2.8,2.8,2.34,2.66,2.9,2.9,3.1,2.91,2.5,2.5,3.18,2.6,2.6,2.6,2.6


In [364]:
df = med_beds_df.drop(["country"], axis = 1)

In [365]:
pred = []
for _, row in df.iterrows():
    auto_arima=pm.auto_arima(row, start_p = 0, start_q = 0, max_p = 12, max_q = 12, m = 12, start_P = 0, start_Q = 0, seasonal = False, error_action = 'ignore', suppress_warnings = True, stepwise = True)
    prediction = pd.DataFrame(auto_arima.predict(n_periods=2))
    pred.append(prediction)

In [366]:
pred_2018 = []
pred_2019 = []
for i in range(len(pred)):
    pred_2018.append(round(pred[i].iat[0, 0], 2))
    pred_2019.append(round(pred[i].iat[1, 0], 2))

In [367]:
med_beds_df["2018"] = pred_2018
med_beds_df["2019"] = pred_2019
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.3,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,...,0.43,0.44,0.53,0.53,0.5,0.5,0.5,0.39,0.39,0.39
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,3.01,...,2.99,2.88,2.88,2.89,2.89,2.89,2.89,2.89,2.87,2.85
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,...,1.93,1.07,1.07,1.15,0.01,1.1,1.36,1.38,1.38,1.38
5,Argentina,4.1,4.0,4.0,4.0,4.0,4.0,4.5,4.5,4.5,...,4.5,4.39,4.59,4.78,4.84,4.96,4.95,4.99,5.04,5.09
6,Armenia,6.44,5.03,4.35,4.42,4.44,4.46,4.42,4.07,3.82,...,3.73,3.74,4.05,4.06,4.16,4.2,4.2,4.2,4.25,4.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,United States,3.49,3.47,3.39,3.33,3.26,3.2,3.18,3.14,3.13,...,3.05,2.97,2.93,2.89,2.83,2.8,2.77,2.87,2.83,2.80
190,Uzbekistan,5.33,5.34,5.54,5.48,5.26,5.19,5.12,4.83,4.67,...,4.44,4.32,4.19,3.99,3.98,3.98,3.98,3.98,3.92,3.84
191,St. Vincent and the Grenadines,4.7,4.7,4.5,4.5,4.5,4.5,3.0,3.0,2.6,...,2.6,2.52,2.41,2.41,2.4,4.33,4.32,4.32,4.23,4.15
194,Vietnam,2.34,2.4,1.4,2.8,2.8,2.34,2.66,2.9,2.9,...,2.91,2.5,2.5,3.18,2.6,2.6,2.6,2.6,2.62,2.62


# Summary

In [383]:
child_mortality_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,129.00,125.00,121.00,117.00,113.00,109.00,104.00,100.00,96.00,...,88.00,84.10,80.50,77.00,73.50,70.40,67.50,64.70,62.30,60.10
1,Angola,206.00,200.00,193.00,185.00,176.00,167.00,157.00,148.00,138.00,...,120.00,113.00,105.00,98.40,92.90,88.10,84.00,80.40,77.20,74.50
2,Albania,25.90,24.50,23.10,21.80,20.40,19.20,17.90,16.70,15.50,...,13.30,12.30,11.40,10.70,10.10,9.60,9.27,9.02,8.82,8.68
3,Andorra,6.41,6.16,5.93,5.71,5.49,5.27,5.05,4.84,4.62,...,4.18,3.98,3.79,3.61,3.43,3.27,3.14,3.01,2.89,2.87
4,United Arab Emirates,11.20,10.90,10.60,10.30,10.00,9.73,9.44,9.18,8.93,...,8.48,8.28,8.13,8.00,7.91,7.83,7.76,7.68,7.59,7.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Samoa,21.10,20.40,19.90,19.50,19.20,19.00,18.90,18.90,18.90,...,18.70,18.50,18.30,17.90,17.60,17.20,16.70,16.30,15.80,15.40
193,Yemen,94.90,90.30,85.60,81.10,76.70,72.50,68.40,64.60,61.00,...,56.00,55.10,55.00,55.00,55.00,55.00,55.00,55.00,55.00,54.90
194,South Africa,73.90,75.80,77.40,79.10,79.40,78.50,76.00,71.00,64.80,...,52.50,46.70,43.30,41.20,39.50,37.70,36.60,35.30,33.80,32.90
195,Zambia,162.00,153.00,142.00,130.00,119.00,110.00,101.00,95.40,90.40,...,80.30,77.20,74.30,70.70,66.90,64.60,62.00,59.40,57.80,56.10


In [384]:
coal_consumption_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United Arab Emirates,0.00000,0.00560,0.02710,0.02990,0.03090,0.03170,0.05310,0.02200,0.04840,...,0.07700,0.04970,0.15100,0.19200,0.21400,0.18500,0.19700,0.22400,0.25000,0.25200
1,Argentina,0.01750,0.01420,0.01050,0.01260,0.02130,0.02420,0.02170,0.03110,0.03760,...,0.02830,0.03040,0.02820,0.03040,0.03210,0.03160,0.02410,0.02450,0.02660,0.01220
2,Australia,2.68000,2.62000,2.65000,2.70000,2.78000,2.72000,2.75000,2.67000,2.73000,...,2.36000,2.26000,2.08000,1.95000,1.90000,1.94000,1.91000,1.82000,1.77000,1.69000
3,Austria,0.45300,0.46600,0.47100,0.50000,0.48700,0.48400,0.48200,0.46400,0.45700,...,0.40100,0.41000,0.37900,0.38300,0.34800,0.37400,0.34400,0.35300,0.32100,0.33600
4,Azerbaijan,0.00000,0.00037,0.00024,0.00036,0.00011,0.00038,0.00037,0.00057,0.00042,...,0.00061,0.00054,0.00040,0.00033,0.00019,0.00032,0.00013,0.00013,0.00012,0.00012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,United States,1.92000,1.84000,1.83000,1.84000,1.84000,1.85000,1.80000,1.81000,1.77000,...,1.61000,1.51000,1.32000,1.36000,1.35000,1.16000,1.05000,1.02000,0.97000,0.82300
75,Uzbekistan,0.02860,0.03070,0.03080,0.02130,0.02930,0.03220,0.03280,0.03390,0.03670,...,0.03590,0.03770,0.03950,0.05420,0.05080,0.04510,0.04150,0.04810,0.06250,0.05260
76,Venezuela,0.00546,0.00198,0.00073,0.00169,0.00000,0.00141,0.00922,0.00509,0.00510,...,0.00701,0.00710,0.00719,0.00706,0.00656,0.00454,0.00413,0.00432,0.00398,0.00265
77,Vietnam,0.05870,0.06350,0.07180,0.08280,0.10700,0.11200,0.06770,0.07340,0.13800,...,0.16500,0.19500,0.17900,0.18900,0.22700,0.28300,0.30300,0.30000,0.39700,0.51300


In [385]:
food_supply_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,1790.0,1740.0,1830.0,1890.0,1970.0,1950.0,1970.0,2050.0,2040.0,...,2100.0,2110.0,2100.0,2090.0,2100.0,2040.0,2030.0,2050.0,2040.0,2040.0
1,Angola,1790.0,1830.0,1920.0,1980.0,2030.0,2080.0,2120.0,2170.0,2250.0,...,2350.0,2410.0,2380.0,2470.0,2420.0,2380.0,2340.0,2370.0,2390.0,2423.0
2,Albania,2730.0,2800.0,2860.0,2770.0,2790.0,2870.0,2860.0,2860.0,2950.0,...,3080.0,3130.0,3180.0,3190.0,3270.0,3280.0,3380.0,3390.0,3360.0,3395.0
4,United Arab Emirates,3300.0,3320.0,3360.0,3340.0,3290.0,3210.0,3200.0,3190.0,3150.0,...,3160.0,3210.0,3230.0,3280.0,3400.0,3410.0,3370.0,3340.0,3310.0,3285.0
5,Argentina,3260.0,3210.0,2980.0,3010.0,3030.0,3110.0,3110.0,3150.0,3160.0,...,3160.0,3160.0,3200.0,3230.0,3250.0,3310.0,3250.0,3270.0,3310.0,3310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Samoa,2770.0,2820.0,2820.0,2830.0,2880.0,2890.0,2860.0,2850.0,2910.0,...,2910.0,2880.0,2960.0,2960.0,2970.0,2970.0,3000.0,3040.0,3030.0,3055.0
174,Yemen,2090.0,2070.0,2080.0,2070.0,2070.0,2090.0,2120.0,2160.0,2130.0,...,2200.0,2210.0,2230.0,2220.0,2160.0,2170.0,2010.0,2070.0,2150.0,2142.0
176,South Africa,2890.0,2910.0,2910.0,2930.0,2940.0,2950.0,2930.0,2920.0,2920.0,...,2990.0,3000.0,3010.0,3020.0,2980.0,2940.0,2920.0,2940.0,2900.0,2892.0
177,Zambia,1870.0,1850.0,1850.0,1900.0,1870.0,1870.0,1840.0,1780.0,1800.0,...,1900.0,1910.0,1920.0,1930.0,2010.0,2030.0,1990.0,1990.0,2000.0,2000.0


In [386]:
med_beds_df

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.3,0.39,0.39,0.39,0.39,0.42,0.42,0.42,0.42,...,0.43,0.44,0.53,0.53,0.5,0.5,0.5,0.39,0.39,0.39
2,Albania,3.26,3.26,3.14,3.07,3.01,3.08,3.12,3.09,3.01,...,2.99,2.88,2.88,2.89,2.89,2.89,2.89,2.89,2.87,2.85
4,United Arab Emirates,2.38,2.28,2.19,2.19,2.19,2.19,1.88,1.88,1.86,...,1.93,1.07,1.07,1.15,0.01,1.1,1.36,1.38,1.38,1.38
5,Argentina,4.1,4.0,4.0,4.0,4.0,4.0,4.5,4.5,4.5,...,4.5,4.39,4.59,4.78,4.84,4.96,4.95,4.99,5.04,5.09
6,Armenia,6.44,5.03,4.35,4.42,4.44,4.46,4.42,4.07,3.82,...,3.73,3.74,4.05,4.06,4.16,4.2,4.2,4.2,4.25,4.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,United States,3.49,3.47,3.39,3.33,3.26,3.2,3.18,3.14,3.13,...,3.05,2.97,2.93,2.89,2.83,2.8,2.77,2.87,2.83,2.80
190,Uzbekistan,5.33,5.34,5.54,5.48,5.26,5.19,5.12,4.83,4.67,...,4.44,4.32,4.19,3.99,3.98,3.98,3.98,3.98,3.92,3.84
191,St. Vincent and the Grenadines,4.7,4.7,4.5,4.5,4.5,4.5,3.0,3.0,2.6,...,2.6,2.52,2.41,2.41,2.4,4.33,4.32,4.32,4.23,4.15
194,Vietnam,2.34,2.4,1.4,2.8,2.8,2.34,2.66,2.9,2.9,...,2.91,2.5,2.5,3.18,2.6,2.6,2.6,2.6,2.62,2.62
